# The Fisher Two-Period Optimal Consumption Problem

[![badge](https://img.shields.io/badge/Launch%20using%20-Econ--ARK-blue)](https://econ-ark.org/materials/fishertwoperiod#launch)

In [1]:
# Some initial setup
import ipywidgets as widgets
from ipywidgets import interact, fixed
from HARK.ConsumptionSaving.ConsIndShockModel import PerfForesightConsumerType
import numpy as np
from copy import deepcopy
from matplotlib import pyplot as plt

plt.style.use("seaborn-darkgrid")
palette = plt.get_cmap("Dark2")


def mystr(number):
    return "{:.3f}".format(number)

This notebook creates interactive widgets illustrating a two-period optimal consumption problem. It also presents graphic representations of the decomposition of the effect of interest rates into the income, substitution, and human wealth effects.

### Basic Plot: the optimal $(c_1,c_2)$ bundle.

In [2]:
# The first step in creating a widget is defining
# a function that will receive the user's input as
# parameters and generate the entity that we want
# to analyze: in this case, a figure of the optimmal
# consumption bundle given income, assets, and interest
# rates


def FisherPlot(Y1, Y2, B1, R, c1Max, c2Max):
    # Basic setup of perfect foresight consumer

    # We first create an instance of the class
    # PerfForesightConsumerType, with its standard parameters.
    PFexample = PerfForesightConsumerType()

    PFexample.cycles = 1  # let the agent live the cycle of periods just once
    PFexample.T_cycle = 2  # Number of periods in the cycle
    # No automatic growth in income across periods
    PFexample.PermGroFac = [1.0]
    PFexample.LivPrb = [1.0]  # No chance of dying before the second period
    PFexample.aNrmInitStd = 0.0
    PFexample.AgentCount = 1
    CRRA = PFexample.CRRA
    Beta = PFexample.DiscFac

    # Set interest rate and bank balances from input.
    PFexample.Rfree = R
    PFexample.aNrmInitMean = B1

    # Solve the model: this generates the optimal consumption function.

    # Try-except blocks "try" to execute the code in the try block. If an
    # error occurs, the except block is executed and the application does
    # not halt
    try:
        PFexample.solve()
    except:
        print("Those parameter values violate a condition required for solution!")

    # Create the figure
    c1Min = 0.0
    c2Min = 0.0
    plt.figure(figsize=(8, 8))
    plt.xlabel("Period 1 Consumption $C_1$")
    plt.ylabel("Period 2 Consumption $C_2$")
    plt.ylim([c2Min, c2Max])
    plt.xlim([c1Min, c1Max])

    # Plot the budget constraint
    C1_bc = np.linspace(c1Min, B1 + Y1 + Y2 / R, 10, endpoint=True)
    C2_bc = (Y1 + B1 - C1_bc) * R + Y2
    plt.plot(C1_bc, C2_bc, "k-", label="Budget Constraint")

    # Plot the optimal consumption bundle
    C1 = PFexample.solution[0].cFunc(B1 + Y1 + Y2 / R)
    C2 = PFexample.solution[1].cFunc((Y1 + B1 - C1) * R + Y2)
    plt.plot(C1, C2, "ro", label="Optimal Consumption")

    # Plot the indifference curve
    V = C1 ** (1 - CRRA) / (1 - CRRA) + Beta * C2 ** (1 - CRRA) / (
        1 - CRRA
    )  # Get max utility
    C1_V = np.linspace(((1 - CRRA) * V) ** (1 / (1 - CRRA)) + 0.5, c1Max, 1000)
    C2_V = (((1 - CRRA) * V - C1_V ** (1 - CRRA)) / Beta) ** (1 / (1 - CRRA))
    plt.plot(C1_V, C2_V, "b-", label="Indiferrence Curve")

    # Add a legend and display the plot
    plt.legend()
    plt.show()

    return None

In [3]:
# We now define the controls that will receive and transmit the
# user's input.
# These are sliders for which the range, step, display, and
# behavior are defined.

# Define a slider for the interest rate
Rfree_widget = widgets.FloatSlider(
    min=1.0,
    max=2.0,
    step=0.001,
    value=1.05,
    continuous_update=True,
    readout_format=".2f",
    description="$R$",
)

# Define a slider for Y1
Y1_widget = widgets.FloatSlider(
    min=0.0,
    max=100.0,
    step=0.1,
    value=50.0,
    continuous_update=True,
    readout_format=".1f",
    description="$Y_1$",
)

# Define a slider for Y2
Y2_widget = widgets.FloatSlider(
    min=0.0,
    max=100.0,
    step=0.1,
    value=50.0,
    continuous_update=True,
    readout_format=".1f",
    description="$Y_2$",
)

# Define a slider for B1
B1_widget = widgets.FloatSlider(
    min=0.0,
    max=4.0,
    step=0.01,
    value=2.0,
    continuous_update=True,
    readout_format=".2f",
    description="$B_1$",
)

# Define a textbox for C1 max
c1Max_widget = widgets.FloatText(
    value=120, step=1.0, description="$C_1$ max", disabled=False
)

# Define a textbox for C2 max
c2Max_widget = widgets.FloatText(
    value=120, step=1.0, description="$C_2$ max", disabled=False
)

In [4]:
# Make the widget
interact(
    FisherPlot,
    Y1=Y1_widget,
    Y2=Y2_widget,
    B1=B1_widget,
    R=Rfree_widget,
    c1Max=c1Max_widget,
    c2Max=c2Max_widget,
)

interactive(children=(FloatSlider(value=50.0, description='$Y_1$', readout_format='.1f'), FloatSlider(value=50…

<function __main__.FisherPlot(Y1, Y2, B1, R, c1Max, c2Max)>

### Second plot: interest rate shifts with lifetime income earned in first period.

In [5]:
# This follows the same process as the previous plot, but now the problem
# is solved at two different interest rates in order to illustrate their effect.


# Define a function that plots something given some bits
def FisherPlot1(Y1, Y2, B1, RInit, RNew, c1Max, c2Max):
    # Basic setup of perfect foresight consumer
    PFexample = (
        PerfForesightConsumerType()
    )  # set up a consumer type and use default parameteres
    PFexample.cycles = 1  # let the agent live the cycle of periods just once
    PFexample.T_cycle = 2  # Number of periods in the cycle
    # No automatic growth in income across periods
    PFexample.PermGroFac = [1.0]
    PFexample.LivPrb = [1.0]  # No chance of dying before the second period
    PFexample.aNrmInitStd = 0.0
    PFexample.AgentCount = 1
    CRRA = 2.0
    Beta = PFexample.DiscFac

    # Set the parameters we enter
    PFexample.aNrmInitMean = B1

    # Create two models, one for RfreeHigh and one for RfreeLow
    PFexampleRInit = deepcopy(PFexample)
    PFexampleRInit.Rfree = RInit
    PFexampleRNew = deepcopy(PFexample)
    PFexampleRNew.Rfree = RNew

    c1Min = 0.0
    c2Min = 0.0

    # Solve the model for RfreeHigh and RfreeLow
    try:
        PFexampleRInit.solve()
        PFexampleRNew.solve()
    except:
        print("Those parameter values violate a condition required for solution!")

    # Plot the chart
    plt.figure(figsize=(8, 8))
    plt.xlabel("Period 1 Consumption $C_1$")
    plt.ylabel("Period 2 Consumption $C_2$")
    plt.ylim([c2Min, c2Max])
    plt.xlim([c1Min, c1Max])

    # Plot the budget constraints
    C1_bc_RNew = np.linspace(c1Min, B1 + Y1 + Y2 / RNew, 10, endpoint=True)
    C2_bc_RNew = (Y1 + B1 - C1_bc_RNew) * RNew + Y2
    plt.plot(C1_bc_RNew, C2_bc_RNew, "k-", label="Budget Constraint R New")

    C1_bc_RInit = np.linspace(c1Min, B1 + Y1 + Y2 / RInit, 10, endpoint=True)
    C2_bc_RInit = (Y1 + B1 - C1_bc_RInit) * RInit + Y2
    plt.plot(C1_bc_RInit, C2_bc_RInit, "k--", label="Budget Constraint R Initial")

    # The optimal consumption bundles
    C1_opt_RNew = PFexampleRNew.solution[0].cFunc(B1 + Y1 + Y2 / RNew)
    C2_opt_RNew = PFexampleRNew.solution[1].cFunc((Y1 + B1 - C1_opt_RNew) * RNew + Y2)

    C1_opt_RInit = PFexampleRInit.solution[0].cFunc(B1 + Y1 + Y2 / RInit)
    C2_opt_RInit = PFexampleRInit.solution[1].cFunc((Y1 + B1 - C1_opt_RInit) * RInit + Y2)

    # Plot the indifference curves
    V_RNew = C1_opt_RNew ** (1 - CRRA) / (1 - CRRA) + Beta * C2_opt_RNew ** (1 - CRRA) / (
        1 - CRRA
    )  # Get max utility
    C1_V_RNew = np.linspace(((1 - CRRA) * V_RNew) ** (1 / (1 - CRRA)) + 0.5, c1Max, 1000)
    C2_V_RNew = (((1 - CRRA) * V_RNew - C1_V_RNew ** (1 - CRRA)) / Beta) ** (
        1 / (1 - CRRA)
    )
    plt.plot(C1_V_RNew, C2_V_RNew, "b-", label="Indiferrence Curve R New")

    V_RInit = C1_opt_RInit ** (1 - CRRA) / (1 - CRRA) + Beta * C2_opt_RInit ** (1 - CRRA) / (
        1 - CRRA
    )  # Get max utility
    C1_V_RInit = np.linspace(((1 - CRRA) * V_RInit) ** (1 / (1 - CRRA)) + 0.5, c1Max, 1000)
    C2_V_RInit = (((1 - CRRA) * V_RInit - C1_V_RInit ** (1 - CRRA)) / Beta) ** (
        1 / (1 - CRRA)
    )
    plt.plot(C1_V_RInit, C2_V_RInit, "b--", label="Indiferrence Curve R Initial")

    # The substitution effect
    C1_Subs = (
        V_RInit * (1 - CRRA) / (1 + Beta * (RNew * Beta) ** ((1 - CRRA) / CRRA))
    ) ** (1 / (1 - CRRA))
    C2_Subs = C1_Subs * (RNew * Beta) ** (1 / CRRA)

    C1_bc_Subs = np.linspace(c1Min, B1 + Y1 + Y2 / RInit, 10, endpoint=True)
    C2_bc_Subs = (C1_Subs + C2_Subs / RNew - C1_bc_Subs) * RNew + Y2
    plt.plot(C1_bc_Subs, C2_bc_Subs, "k-")

    # Plot the points of interest
    plt.plot(C1_opt_RNew, C2_opt_RNew, "ro", label="A: Optimal Consumption R New")
    plt.plot(
        C1_Subs,
        C2_Subs,
        "go",
        label="B: Income effect AB \n     Substitution effect BC ",
    )
    plt.plot(C1_opt_RInit, C2_opt_RInit, "mo", label="C: Optimal Consumption R Initial")

    plt.legend()
    plt.show()

    return None

In [6]:
# Define some widgets to control the plot

# Define a slider for the high interest rate
RInit_widget1 = widgets.FloatSlider(
    min=1.0,
    max=4.0,
    step=0.01,
    value=2,
    continuous_update=True,
    readout_format=".2f",
    description="$R_{Initial}$",
)

# Define a slider for the low interest rate
RNew_widget1 = widgets.FloatSlider(
    min=0.0,
    max=4.0,
    step=0.01,
    value=1.0,
    continuous_update=True,
    readout_format=".2f",
    description="$R_{New}$",
)

# Define a slider for Y1
Y1_widget1 = widgets.FloatSlider(
    min=0.0,
    max=100.0,
    step=0.1,
    value=100.0,
    continuous_update=True,
    readout_format=".1f",
    description="$Y_1$",
)

# Define a slider for Y2
Y2_widget1 = widgets.FloatSlider(
    min=0.0,
    max=100.0,
    step=0.1,
    value=0.0,
    continuous_update=True,
    readout_format=".1f",
    description="$Y_2$",
)

# Define a slider for B1
B1_widget1 = widgets.FloatSlider(
    min=0.0,
    max=4.0,
    step=0.01,
    value=0.0,
    continuous_update=True,
    readout_format=".2f",
    description="$B_1$",
)

# Define a textbox for C1 max
c1Max_widget1 = widgets.FloatText(
    value=120, step=1.0, description="$C1$ max", disabled=False
)

# Define a textbox for C2 max
c2Max_widget1 = widgets.FloatText(
    value=120, step=1.0, description="$C2$ max", disabled=False
)

In [7]:
# Make the widget

interact(
    FisherPlot1,
    Y1=Y1_widget1,
    Y2=fixed(0.0),
    B1=B1_widget1,
    RInit=RInit_widget1,
    RNew=RNew_widget1,
    c1Max=c1Max_widget1,
    c2Max=c2Max_widget1,
)

interactive(children=(FloatSlider(value=100.0, description='$Y_1$', readout_format='.1f'), FloatSlider(value=0…

<function __main__.FisherPlot1(Y1, Y2, B1, RInit, RNew, c1Max, c2Max)>

### Third plot: interest rate shifts with lifetime income earned in second period

In [8]:
# This follows the same process, but we now fix Y_1 at 0


# Define a function that plots something given some bits
def FisherPlot2(Y1, Y2, B1, RInit, RNew, c1Max, c2Max):
    # Basic setup of perfect foresight consumer
    PFexample = (
        PerfForesightConsumerType()
    )  # set up a consumer type and use default parameteres
    PFexample.cycles = 1  # let the agent live the cycle of periods just once
    PFexample.T_cycle = 2  # Number of periods in the cycle
    # No automatic growth in income across periods
    PFexample.PermGroFac = [1.0]
    PFexample.LivPrb = [1.0]  # No chance of dying before the second period
    PFexample.aNrmInitStd = 0.0
    PFexample.AgentCount = 1
    CRRA = 2.0
    Beta = PFexample.DiscFac

    # Set the parameters we enter
    PFexample.aNrmInitMean = B1

    # Create two models, one for RfreeHigh and one for RfreeLow
    PFexampleRInit = deepcopy(PFexample)
    PFexampleRInit.Rfree = RInit
    PFexampleRNew = deepcopy(PFexample)
    PFexampleRNew.Rfree = RNew

    c1Min = 0.0
    c2Min = 0.0

    # Solve the model for RfreeHigh
    try:
        PFexampleRInit.solve()
        PFexampleRNew.solve()
    except:
        print("Those parameter values violate a condition required for solution!")

    # Plot the chart
    plt.figure(figsize=(8, 8))
    plt.xlabel("Period 1 Consumption $C_1$")
    plt.ylabel("Period 2 Consumption $C_2$")
    plt.ylim([c2Min, c2Max])
    plt.xlim([c1Min, c1Max])

    # Plot the budget constraints
    C1_bc_RNew = np.linspace(c1Min, B1 + Y1 + Y2 / RNew, 10, endpoint=True)
    C2_bc_RNew = (Y1 + B1 - C1_bc_RNew) * RNew + Y2
    plt.plot(C1_bc_RNew, C2_bc_RNew, "k-", label="Budget Constraint R New")

    C1_bc_RInit = np.linspace(c1Min, B1 + Y1 + Y2 / RInit, 10, endpoint=True)
    C2_bc_RInit = (Y1 + B1 - C1_bc_RInit) * RInit + Y2
    plt.plot(C1_bc_RInit, C2_bc_RInit, "k--", label="Budget Constraint R Initial")

    # The optimal consumption bundles
    C1_opt_RNew = PFexampleRNew.solution[0].cFunc(B1 + Y1 + Y2 / RNew)
    C2_opt_RNew = PFexampleRNew.solution[1].cFunc((Y1 + B1 - C1_opt_RNew) * RNew + Y2)

    C1_opt_RInit = PFexampleRInit.solution[0].cFunc(B1 + Y1 + Y2 / RInit)
    C2_opt_RInit = PFexampleRInit.solution[1].cFunc((Y1 + B1 - C1_opt_RInit) * RInit + Y2)

    # Plot the indifference curves
    V_RNew = C1_opt_RNew ** (1 - CRRA) / (1 - CRRA) + Beta * C2_opt_RNew ** (1 - CRRA) / (
        1 - CRRA
    )  # Get max utility
    C1_V_RNew = np.linspace(((1 - CRRA) * V_RNew) ** (1 / (1 - CRRA)) + 0.5, c1Max, 1000)
    C2_V_RNew = (((1 - CRRA) * V_RNew - C1_V_RNew ** (1 - CRRA)) / Beta) ** (
        1 / (1 - CRRA)
    )
    plt.plot(C1_V_RNew, C2_V_RNew, "b-", label="Indiferrence Curve R New")

    V_RInit = C1_opt_RInit ** (1 - CRRA) / (1 - CRRA) + Beta * C2_opt_RInit ** (1 - CRRA) / (
        1 - CRRA
    )  # Get max utility
    C1_V_RInit = np.linspace(((1 - CRRA) * V_RInit) ** (1 / (1 - CRRA)) + 0.5, c1Max, 1000)
    C2_V_RInit = (((1 - CRRA) * V_RInit - C1_V_RInit ** (1 - CRRA)) / Beta) ** (
        1 / (1 - CRRA)
    )
    plt.plot(C1_V_RInit, C2_V_RInit, "b--", label="Indiferrence Curve R Initial")

    # The substitution effect
    C1_Subs = (
        V_RInit * (1 - CRRA) / (1 + Beta * (RNew * Beta) ** ((1 - CRRA) / CRRA))
    ) ** (1 / (1 - CRRA))
    C2_Subs = C1_Subs * (RNew * Beta) ** (1 / CRRA)

    # The Human wealth effect
    Y2HW = Y2 * RNew / RInit
    C1HW = Y2HW / (RNew + (RNew) ** (1 / CRRA))
    C2HW = C1HW * (RNew * Beta) ** (1 / CRRA)

    C1_bc_HW = np.linspace(c1Min, B1 + Y1 + Y2HW / RNew, 10, endpoint=True)
    C2_bc_HW = (Y1 + B1 - C1_bc_HW) * RNew + Y2HW
    plt.plot(C1_bc_HW, C2_bc_HW, "k:")

    VHW = (C1HW ** (1 - CRRA)) / (1 - CRRA) + (Beta * C2HW ** (1 - CRRA)) / (1 - CRRA)
    C1_V_HW = np.linspace(((1 - CRRA) * VHW) ** (1 / (1 - CRRA)) + 0.5, c1Max, 1000)
    C2_V_HW = (((1 - CRRA) * VHW - C1_V_HW ** (1 - CRRA)) / Beta) ** (1 / (1 - CRRA))
    plt.plot(C1_V_HW, C2_V_HW, "b:")

    # Plot the points of interest
    plt.plot(C1_opt_RNew, C2_opt_RNew, "ro", label="A: Optimal Consumption R New")
    plt.plot(
        C1_Subs, C2_Subs, "go", label="B: Income effect DB \n    Substitution effect BC"
    )
    plt.plot(C1_opt_RInit, C2_opt_RInit, "mo", label="C: Optimal Consumption R Initial")
    plt.plot(C1HW, C2HW, "co", label="D: HW effect AD")

    plt.legend()
    plt.show()

    return None

In [9]:
# Define some widgets to control the plot

# Define a slider for the high interest rate
RInit_widget2 = widgets.FloatSlider(
    min=1.0,
    max=4.0,
    step=0.01,
    value=2,
    continuous_update=True,
    readout_format=".2f",
    description="$R_{Initial}$",
)

# Define a slider for the low interest rate
RNew_widget2 = widgets.FloatSlider(
    min=0.0,
    max=4.0,
    step=0.01,
    value=1.0,
    continuous_update=True,
    readout_format=".2f",
    description="$R_{New}$",
)

# Define a slider for Y1
Y1_widget2 = widgets.FloatSlider(
    min=0.0,
    max=100.0,
    step=0.1,
    value=100.0,
    continuous_update=True,
    readout_format=".1f",
    description="$Y_1$",
)

# Define a slider for Y2
Y2_widget2 = widgets.FloatSlider(
    min=0.0,
    max=100.0,
    step=0.1,
    value=100.0,
    continuous_update=True,
    readout_format=".1f",
    description="$Y_2$",
)

# Define a slider for B1
B1_widget2 = widgets.FloatSlider(
    min=0.0,
    max=4.0,
    step=0.01,
    value=0.0,
    continuous_update=True,
    readout_format=".1f",
    description="$B_1$",
)

# Define a textbox for C1 max
c1Max_widget2 = widgets.FloatText(
    value=120, step=1.0, description="$C_1$ max", disabled=False
)

# Define a textbox for C2 max
c2Max_widget2 = widgets.FloatText(
    value=120, step=1.0, description="$C_2$ max", disabled=False
)

In [10]:
# Make the widget

interact(
    FisherPlot2,
    Y1=fixed(0.0),
    Y2=Y2_widget2,
    B1=fixed(0.0),
    RInit=RInit_widget2,
    RNew=RNew_widget2,
    c1Max=c1Max_widget2,
    c2Max=c2Max_widget2,
)

interactive(children=(FloatSlider(value=100.0, description='$Y_2$', readout_format='.1f'), FloatSlider(value=2…

<function __main__.FisherPlot2(Y1, Y2, B1, RInit, RNew, c1Max, c2Max)>

# Fourth plot: the effects of changes in the coefficient of risk aversion

For this exercise, we assume that no income is received in the second period. The relevant parameter is therefore $M_1$, the total market resources before consumption in period 1.

In [11]:
# Create plotting function
def FisherPlot3(M1, R, Beta, CRRA, c1Max, c2Max):
    # Basic setup of perfect foresight consumer

    # We first create an instance of the class
    # PerfForesightConsumerType, with its standard parameters.
    PFexample = PerfForesightConsumerType()

    PFexample.cycles = 1  # let the agent live the cycle of periods just once
    PFexample.T_cycle = 1  # One single non-terminal period
    PFexample.PermGroFac = [0.00001] # No income in the second period (Using 0.0 leads to errors)
    PFexample.LivPrb = [1.0]  # No chance of dying before the second period

    # Set interest rate and bank balances from input.
    PFexample.Rfree = R
    PFexample.CRRA = CRRA
    PFexample.DiscFac = Beta

    # Solve the model: this generates the optimal consumption function.

    # Try-except blocks "try" to execute the code in the try block. If an
    # error occurs, the except block is executed and the application does
    # not halt
    try:
        PFexample.solve()
    except:
        print("Those parameter values violate a condition required for solution!")
        return

    # Create the figure
    c1Min = 0.0
    c2Min = 0.0
    plt.figure(figsize=(8, 8))
    plt.xlabel("Period 1 Consumption $C_1$")
    plt.ylabel("Period 2 Consumption $C_2$")
    plt.ylim([c2Min, c2Max])
    plt.xlim([c1Min, c1Max])

    # Plot the budget constraint
    C1_bc = np.linspace(c1Min, M1, 10, endpoint=True)
    C2_bc = (M1 - C1_bc) * R
    plt.plot(C1_bc, C2_bc, "k-", label="Budget Constraint")

    # Plot the optimal consumption bundle
    C1 = PFexample.solution[0].cFunc(M1)
    C2 = PFexample.solution[1].cFunc((M1 - C1) * R)
    plt.plot(C1, C2, "ro", label="Optimal Consumption")

    # Plot the indifference curve
    V = C1 ** (1 - CRRA) / (1 - CRRA) + Beta * C2 ** (1 - CRRA) / (
        1 - CRRA
    )  # Get max utility
    C1_V = np.linspace(((1 - CRRA) * V) ** (1 / (1 - CRRA)) + 0.1, c1Max, 1000)
    C2_V = (((1 - CRRA) * V - C1_V ** (1 - CRRA)) / Beta) ** (1 / (1 - CRRA))
    plt.plot(C1_V, C2_V, "b-", label="Indiferrence Curve")

    # Add a legend and display the plot
    plt.legend()
    plt.show()

    return None

In [12]:
# We now define the controls that will receive and transmit the
# user's input.
# These are sliders for which the range, step, display, and
# behavior are defined.

# Define a slider for the interest rate
Rfree_widget = widgets.FloatSlider(
    min=1.0,
    max=2.0,
    step=0.01,
    value=1.05,
    continuous_update=True,
    readout_format=".2f",
    description="$R$",
)

# Define a slider for the discount factor
Beta_widget = widgets.FloatSlider(
    min=0.5,
    max=1.0,
    step=0.01,
    value=0.9,
    continuous_update=True,
    readout_format=".2f",
    description="$\\beta$",
)

# Define a slider for the CRRA
CRRA_widget = widgets.FloatSlider(
    min=1.1,
    max=4.0,
    step=0.1,
    value=2,
    continuous_update=True,
    readout_format=".1f",
    description="$\\rho$",
)

# Define a slider for M1
M1_widget = widgets.FloatSlider(
    min=0.0,
    max=20.0,
    step=0.1,
    value=10.0,
    continuous_update=True,
    readout_format=".1f",
    description="$M_1$",
)

# Define a textbox for C1 max
c1Max_widget = widgets.FloatText(
    value=20, step=1.0, description="$C_1$ max", disabled=False
)

# Define a textbox for C2 max
c2Max_widget = widgets.FloatText(
    value=20, step=1.0, description="$C_2$ max", disabled=False
)

# Make the widget
interact(
    FisherPlot3,
    M1=M1_widget,
    R=Rfree_widget,
    Beta=Beta_widget,
    CRRA=CRRA_widget,
    c1Max=c1Max_widget,
    c2Max=c2Max_widget,
)

interactive(children=(FloatSlider(value=10.0, description='$M_1$', max=20.0, readout_format='.1f'), FloatSlide…

<function __main__.FisherPlot3(M1, R, Beta, CRRA, c1Max, c2Max)>